In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import pickle
import csv
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score, accuracy_score
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut
from xgboost import XGBClassifier

## 데이터 전처리

In [2]:
df_jinhyeok = pd.read_csv("./data/weather_area_op.csv")
df_jinhyeok.rename(columns={"Unnamed: 0":"행정구역","Unnamed: 1":"계절","Unnamed: 2":"연도"},inplace=True)

month_spring = ["_3","_4","_5"]           # ["_3","_4","_5"]
month_summer = ["_6","_7","_8"]      # ["_5","_6","_7","_8"]
month_fall = ["_8","_9","_10"]      # ["_8","_9","_10","_11"]
month_winter = ["_9","_10","_11"]      # ["_8","_9","_10","_11","_12","_1","_2"]

season = ["spring", "summer", "fall", "winter"]

for i in range(len(season)) : 
    globals()["df_"+season[i]] = df_jinhyeok[df_jinhyeok["계절"]==season[i]]  # 계절별로 데이터 프레임 형성
    globals()["df_"+season[i]] = globals()["df_"+season[i]].groupby(["행정구역","계절","연도"]).mean()   # 행정구역, 계절, 연도를 인덱스로 groupby

In [3]:
with open("./data/weather_area_op.csv", "r", encoding="utf8") as f: 
    dr = csv.reader(f)
    s = pd.DataFrame(dr)
    s = s.rename(columns=s.iloc[0])
    s.drop(index=s.index[0], axis=0,inplace=True)

    s.columns = ["지역", '계절', '연도', '평균기온(°C)_1월', '최고기온(°C)_1월', '최저기온(°C)_1월',
       '월합강수량(00~24h만)(mm)_1월', '합계 일사량(MJ/m2)_1월', '평균기온(°C)_2월',
       '최고기온(°C)_2월', '최저기온(°C)_2월', '월합강수량(00~24h만)(mm)_2월',
       '합계 일사량(MJ/m2)_2월', '평균기온(°C)_3월', '최고기온(°C)_3월', '최저기온(°C)_3월',
       '월합강수량(00~24h만)(mm)_3월', '합계 일사량(MJ/m2)_3월', '평균기온(°C)_4월',
       '최고기온(°C)_4월', '최저기온(°C)_4월', '월합강수량(00~24h만)(mm)_4월',
       '합계 일사량(MJ/m2)_4월', '평균기온(°C)_5월', '최고기온(°C)_5월', '최저기온(°C)_5월',
       '월합강수량(00~24h만)(mm)_5월', '합계 일사량(MJ/m2)_5월', '평균기온(°C)_6월',
       '최고기온(°C)_6월', '최저기온(°C)_6월', '월합강수량(00~24h만)(mm)_6월',
       '합계 일사량(MJ/m2)_6월', '평균기온(°C)_7월', '최고기온(°C)_7월', '최저기온(°C)_7월',
       '월합강수량(00~24h만)(mm)_7월', '합계 일사량(MJ/m2)_7월', '평균기온(°C)_8월',
       '최고기온(°C)_8월', '최저기온(°C)_8월', '월합강수량(00~24h만)(mm)_8월',
       '합계 일사량(MJ/m2)_8월', '평균기온(°C)_9월', '최고기온(°C)_9월', '최저기온(°C)_9월',
       '월합강수량(00~24h만)(mm)_9월', '합계 일사량(MJ/m2)_9월', '평균기온(°C)_10월',
       '최고기온(°C)_10월', '최저기온(°C)_10월', '월합강수량(00~24h만)(mm)_10월',
       '합계 일사량(MJ/m2)_10월', '평균기온(°C)_11월', '최고기온(°C)_11월', '최저기온(°C)_11월',
       '월합강수량(00~24h만)(mm)_11월', '합계 일사량(MJ/m2)_11월', '평균기온(°C)_12월',
       '최고기온(°C)_12월', '최저기온(°C)_12월', '월합강수량(00~24h만)(mm)_12월',
       '합계 일사량(MJ/m2)_12월', '면적 (ha)', '생산량 (톤)']

In [4]:
spr = "|".join(w for w in month_spring)

In [5]:
for i in range(len(season)) : 
    globals()["month_str_"+season[i]] = "|".join(w for w in globals()["month_"+season[i]])  # 각 계절별 월을 | 으로 join
    
    # 각 계절이 속하는 월과 면적, 생산이 속하는 컬럼명을 리스트로 저장
    globals()["col_of_"+season[i]] = globals()["df_"+season[i]].columns[globals()["df_"+season[i]].columns.str.contains(globals()["month_str_"+season[i]]+"|면적|생산")]

In [6]:
# 데이터프레임을 계절별로 분리
df_spring = df_spring[col_of_spring]
df_summer = df_summer[col_of_summer]
df_fall = df_fall[col_of_fall]
df_winter = df_winter[col_of_winter]

In [7]:
df_winter.dropna(how="any",inplace=True)
# df_winter

In [8]:
city = ["서울특별시", "부산광역시", "대구광역시", "인천광역시", "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", "경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "제주도"]
city = sorted(city)
# city

In [9]:
# df_winter[df_winter["생산량 (톤)"].isnull()]

In [10]:
# 생산량이 NaN인 행을 drop
df_spring.dropna(how="any", inplace=True)
df_summer.dropna(how="any", inplace=True)
df_fall.dropna(how="any", inplace=True)

In [11]:
# 면적 컬럼 삭제
for s in season : 
    globals()["df_"+s].drop(columns="면적 (ha)", inplace=True)

In [12]:
# 배추 생산 시기를 임의로 상, 중, 하로 나누었으며, 거기에는 육묘기, 생육기, 결구기 등이 포함돼 있음
for s in season : 
    globals()["df_"+s].columns = ['평균기온(°C)_상', '최고기온(°C)_상', '최저기온(°C)_상', '월합강수량(00~24h만)(mm)_상',
       '합계 일사량(MJ/m2)_상', '평균기온(°C)_중', '최고기온(°C)_중', '최저기온(°C)_중',
       '월합강수량(00~24h만)(mm)_중', '합계 일사량(MJ/m2)_중', '평균기온(°C)_하',
       '최고기온(°C)_하', '최저기온(°C)_하', '월합강수량(00~24h만)(mm)_하',
       '합계 일사량(MJ/m2)_하', '생산량 (톤)']

In [13]:
df_total = pd.concat([df_spring, df_summer, df_fall, df_winter])
df_total.head()

평균기온(°C)_상  최고기온(°C)_상  최저기온(°C)_상  월합강수량(00~24h만)(mm)_상  \
행정구역 계절     연도                                                               
강원도  spring 2000        4.40       19.30        -8.6                 14.10   
            2001        5.25       19.75        -8.6                  2.55   
            2002        7.30       19.70        -7.3                 33.10   
            2003        6.60       21.30        -6.0                 28.50   
            2004        6.90       20.30        -4.9                 19.90   

                  합계 일사량(MJ/m2)_상  평균기온(°C)_중  최고기온(°C)_중  최저기온(°C)_중  \
행정구역 계절     연도                                                          
강원도  spring 2000           448.65       10.40        23.7        -3.1   
            2001           455.28       10.85        23.7        -2.2   
            2002           519.21       12.70        24.8        -1.2   
            2003           523.00       11.90        25.9        -1.4   
            2004           481.75       12.60        25.4        -0.5   

                  월합강수량(00~24h만)(mm)_중  합계 일사량(MJ/m2)_중  평균기온(°C)_하  \
행정구역 계절     연도                                                        
강원도  spring 2000                 37.10          486.355       15.90   
            2001                 23.55          530.950       17.05   
            2002                 50.00          542.480       17.70   
            2003                 49.00          540.765       17.50   
            2004                 16.30          537.210       17.80   

                  최고기온(°C)_하  최저기온(°C)_하  월합강수량(00~24h만)(mm)_하  \
행정구역 계절     연도                                                   
강원도  spring 2000        31.6        3.90                 75.20   
            2001        31.4        5.55                 67.55   
            2002        34.1        5.70                 65.00   
            2003        34.5        5.60                 52.60   
            2004        33.9        5.80                 47.70   

                  합계 일사량(MJ/m2)_하   생산량 (톤)  
행정구역 계절     연도                               
강원도  spring 2000          520.280   87619.0  
            2001          562.770   83405.0  
            2002          586.320   56908.0  
            2003          586.195   82760.0  
            2004          540.490  100486.0

In [14]:
# df_total[df_total["생산량 (톤)"] == 0]

In [15]:
# 생산량이 0이상인 지역의 기후 조건을 적합, 0인 지역의 기후 조건을 부적합하다고 labeling
df_total["재배 적합"] = df_total["생산량 (톤)"].apply(lambda x : 1 if x > 0 else 0)

In [16]:
# df_total[df_total["재배 적합"] == 1].tail(30)
# df_total.head()

In [17]:
# df_spring[df_spring["생산량 (톤)"]==0]

In [18]:
df_total.drop(columns="생산량 (톤)", inplace=True)
df_total.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1212 entries, ('강원도', 'spring', 2000) to ('충청북도', 'winter', 2021)
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   평균기온(°C)_상            1212 non-null   float64
 1   최고기온(°C)_상            1212 non-null   float64
 2   최저기온(°C)_상            1212 non-null   float64
 3   월합강수량(00~24h만)(mm)_상  1212 non-null   float64
 4   합계 일사량(MJ/m2)_상       1212 non-null   float64
 5   평균기온(°C)_중            1212 non-null   float64
 6   최고기온(°C)_중            1212 non-null   float64
 7   최저기온(°C)_중            1212 non-null   float64
 8   월합강수량(00~24h만)(mm)_중  1212 non-null   float64
 9   합계 일사량(MJ/m2)_중       1212 non-null   float64
 10  평균기온(°C)_하            1212 non-null   float64
 11  최고기온(°C)_하            1212 non-null   float64
 12  최저기온(°C)_하            1212 non-null   float64
 13  월합강수량(00~24h만)(mm)_하  1212 non-null   float64
 14  합계 일사량(MJ/m2)_하       1212 non

In [19]:
# 상관 분석(안 해도 되나)
# plt.rc('font', family='Malgun Gothic')

# corr = np.corrcoef(df_total.values.T)
# fig = plt.figure(figsize=(16,16))
# sns.heatmap(corr, cbar=True, annot=True, square=False, xticklabels=df_total.columns, yticklabels=df_total.columns, annot_kws={"size":7})

## train, test set 분리

In [20]:
X = df_total.iloc[:,:-1]
y = df_total.iloc[:,-1:]

In [20]:
std = StandardScaler()
std.fit(X)
X = std.transform(X)

In [33]:
mmsc = MinMaxScaler()
mmsc.fit(X)
X = std.transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

## GridSearchCV를 통한 최적의 파라미터

In [ ]:
# 교차 검증을 위한 kfold
kfold = KFold(n_splits=10, shuffle=True, random_state=27)
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=27)

alpha = np.logspace(-4,2,7)   # lasso, ridge 해당
learning_rate = np.logspace(-4,2,7)   # adaboost, gradientboost 해당
eta = np.logspace(-4,2,7)   # xgboost 해당
max_iter = [1,3,5,10,20,50,100,150,200]   # lasso, ridge 해당
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']   # ridge 해당, 계산에 사용할 알고리즘
n_estimators = [1,3,5,10,15,20,50,100,200,500]   # randomforesteregressor(사용할 tree의 개수), gradientboost, adaboost, xgboost 해당
max_depth = [2,3,5,7,10,12]  # randomforesteregressor(tree의 깊이), gradientboost, xgboost 해당
criterion_rf = ["gini", "entropy", "log_loss"]  # randomforesteregressor 해당
criterion_gb = ["squared_error", "friedman_mse"]  # gradientboost 해당
algorithm_ada = ["SAMME", "SAMME.R"]    # adaboost 해당
loss_gb = ["log_loss", "deviance", "exponential"]  # gradientboost 해당
gamma = [0, 0.01, 0.05, 0.1, 0.2, 0.5]    # xgboost 해당
max_leaf_nodes = [2,3,5,7]     # randomforest, xgboost 해당
min_child_weight = [1,2,3,4,5]   # xgboost 해당
min_samples_leaf = [1,2,3,4,5]   # randomforesst 해당
min_weight_fraction_leaf = [0, 0.1, 0.2, 0.4, 0.5]  # randomforest 해당
max_features = ["sqrt", "log2"]   # randomforest 해당
max_features_grad = ["auto", "sqrt", "log2"]   # randomforest 해당

params_rf = {"n_estimators" : n_estimators, 
                "max_depth" : max_depth, 
                "criterion" : criterion_rf,
                "max_leaf_nodes" : max_leaf_nodes,
                "min_samples_leaf" : min_samples_leaf,
                "min_weight_fraction_leaf" : min_weight_fraction_leaf,
                "max_features" : max_features}

params_ada = {"learning_rate" : learning_rate, 
                "n_estimators" : n_estimators, 
                "algorithm" : algorithm_ada
                }

params_grad = {"learning_rate" : learning_rate, 
                "n_estimators" : n_estimators, 
                "max_depth" : max_depth,
                "criterion" : criterion_gb,
                "loss" : loss_gb,
                "max_leaf_nodes" : max_leaf_nodes,
                "min_samples_leaf" : min_samples_leaf,
                "min_weight_fraction_leaf" : min_weight_fraction_leaf,
                "max_features" : max_features_grad}

params_xgb = {"learning_rate" : eta,
                "n_estimators" : n_estimators, 
                "max_depth" : max_depth, 
                "gamma" : gamma,
                "max_leaf_nodes" : max_leaf_nodes,
                "min_child_weight" : min_child_weight}



In [ ]:
rf = RandomForestClassifier()
grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_rf.fit(X, y)
print("Best Score :", grid_rf.best_score_)
print("Best Parameters :", grid_rf.best_params_)

KeyboardInterrupt: 

In [ ]:
ada = AdaBoostClassifier()
grid_ada = GridSearchCV(estimator=ada, param_grid=params_ada, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_ada.fit(X, y)
print("Best Score :", grid_ada.best_score_)
print("Best Parameters :", grid_ada.best_params_)

Best Score : -0.11474055006096735
Best Parameters : {'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 100}


In [ ]:
grad = GradientBoostingClassifier()
grid_grad = GridSearchCV(estimator=grad, param_grid=params_grad, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_grad.fit(X, y)
print("Best Score :", grid_grad.best_score_)
print("Best Parameters :", grid_grad.best_params_)

Best Score : -0.08828749491938762
Best Parameters : {'criterion': 'squared_error', 'learning_rate': 1.0, 'loss': 'exponential', 'max_depth': 5, 'n_estimators': 100}


In [ ]:
xgb = XGBClassifier()
grid_xgb = GridSearchCV(estimator=xgb, param_grid=params_xgb, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_xgb.fit(X, y)
print("Best Score :", grid_xgb.best_score_)
print("Best Parameters :", grid_xgb.best_params_)

Best Score : -0.0924129521745021
Best Parameters : {'eta': 0.1, 'max_depth': 5, 'n_estimators': 50}


In [ ]:
"""Best Parameters : {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'max_leaf_nodes': 2, 'min_child_weight': 2, 'n_estimators': 200}"""

## xgboost를 이용한 배추 재배 적합 이진분류

In [ ]:
skfold2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=27)
skfold2.get_n_splits(X,y)

for train_index, test_index in skfold2.split(X, y) : 
    print("Train :", train_index, "Test :", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.values.reshape(-1,1)[train_index], y.values.reshape(-1,1)[test_index]

xgb2 = XGBClassifier(learning_rate=0.05, gamma=0, max_depth=4, min_child_weight=2, n_estimators=200)
xgb2.fit(X_train,y_train)

y_pred = xgb2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [23]:
for train_index, test_index in skfold2.split(X, y) : 
    print("Train :", train_index, "Test :", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.values.reshape(-1,1)[train_index], y.values.reshape(-1,1)[test_index]

Train : [   0    1    2    3    5    6    7    8    9   10   12   13   14   15
   16   17   20   21   22   23   25   26   27   30   31   35   36   37
   39   40   41   42   43   44   45   46   47   48   50   52   53   54
   56   57   58   60   63   64   65   66   67   68   69   70   71   72
   73   74   75   76   77   78   79   80   81   82   84   85   86   87
   88   89   90   91   92   93   94   95   96   97   98   99  100  101
  103  104  105  106  108  109  111  112  115  117  118  119  120  121
  123  124  126  127  129  130  131  132  133  134  136  137  138  140
  141  142  143  144  145  148  149  150  151  152  154  157  158  159
  160  161  162  163  164  166  167  170  171  172  174  175  177  178
  179  180  181  182  183  184  185  186  187  188  189  190  191  192
  193  194  195  196  197  198  200  201  202  203  204  205  206  207
  210  211  212  213  215  217  218  219  220  221  222  224  226  227
  228  229  232  233  234  236  237  238  239  240  241  242  243  24

In [24]:
xgb2 = XGBClassifier(learning_rate=0.05, gamma=0, max_depth=4, min_child_weight=2, n_estimators=200)
xgb2.fit(X_train,y_train)

y_pred = xgb2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=4, max_leaves=0, min_child_weight=2, missing=nan,
              monotone_constraints='()', n_estimators=200, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [25]:
y_pred = xgb2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9132231404958677


In [27]:
pred_test = [7.30,19.80,-1.70,53.9,390.280,14.1,28.00,3.80,38.50,496.970,17.7,28.50,8.10,97.70,560.500]
pred_test = np.array(pred_test).reshape(1,-1)
pred_test = std.transform(pred_test)

y_pred2 = xgb2.predict(pred_test)
print(y_pred2)

[1]


In [28]:
pred_test2 = [5.25,19.75,-8.6,2.55,455.28,10.85,23.,-2.2,23.55,530.950,17.05,31.4,5.55,67.55,562.770]
pred_test2 = np.array(pred_test2).reshape(1,-1)
pred_test2 = std.transform(pred_test2)

y_pred3 = xgb2.predict(pred_test2)
print(y_pred3)

[1]


In [29]:
pred_test3 = [6.60,21.30,-6.0,28.50,523.00,11.90,25.9,-1.4,49.00,540.765,17.50,34.5,5.60,52.60,586.195]
pred_test3 = np.array(pred_test3).reshape(1,-1)
pred_test3 = std.transform(pred_test3)

y_pred4= xgb2.predict(pred_test3)
print(y_pred4)

[1]


In [ ]:
y_pred_train = xgb2.predict(X_train)
print("accuracy_score :", accuracy_score(y_train, y_pred_train))

accuracy_score : 0.9804123711340206


In [28]:
pred_test4 = [25.3,	34.00,	18.90,	358.70,	496.760,	22.10,	30.2,	13.50,	139.90,	415.265,	15.60,	30.60,	0.90,	43.20,	423.275]
pred_test4 = np.array(pred_test4).reshape(1,-1)
pred_test4 = std.transform(pred_test4)

y_pred5= xgb2.predict(pred_test4)
print(y_pred5)

[1]


In [26]:
## 적합 판단 모델 저장
f = open("xgb_baechoo_bin_classify_jinhyeok.pickle", "wb")
pickle.dump(xgb2,f)

In [29]:
## 적합 판단 모델 불러오기
with open('xgb_baechoo_bin_classify_jinhyeok.pickle', 'rb') as f: 
    model = pickle.load(f)
    y_p = model.predict(pred_test4)
    print(y_p)

[1]


In [33]:
## standardscaler 저장
# o = open("xgb_baechoo_bin_classify_scaler_jinhyeok.pickle", "wb")
# pickle.dump(std,o)

# o = open("xgb_baechoo_bin_classify_scaler_jinhyeok.pkl", "wb")
joblib.dump(std,"xgb_baechoo_bin_classify_scaler_jinhyeok.pkl")

['xgb_baechoo_bin_classify_scaler_jinhyeok.pkl']

In [34]:
pred_test4 = [25.3,	34.00,	18.90,	358.70,	496.760,	22.10,	30.2,	13.50,	139.90,	415.265,	15.60,	30.60,	0.90,	43.20,	423.275]
pred_test4 = np.array(pred_test4).reshape(1,-1)

# with open("xgb_baechoo_bin_classify_scaler_jinhyeok.pickle", "rb") as f : 
#     scaler = pickle.load(f)
#     scaling = scaler.transform(pred_test4)
#     print(scaling)
scaler = joblib.load("xgb_baechoo_bin_classify_scaler_jinhyeok.pkl")
scaling = scaler.transform(pred_test4)
print(scaling)

[[ 0.86837142  0.78583699  1.03906174  1.54897321  0.14175334  0.56780096
   0.07846137  0.5301293  -0.2011774  -0.58218877 -0.47677394  0.25288718
  -0.91917529 -0.68473015 -0.33459753]]


## logistic regression을 통한 이진분류

In [22]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print("accuracy :", accuracy_score(y_test, y_pred))

accuracy : 0.7366255144032922


In [23]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("accuracy :", accuracy_score(y_test, y_pred))

accuracy : 0.8847736625514403


In [24]:
# df_summer

In [34]:
test = [15.90,21.80,12.20,133.50,471.260,17.80,25.10,18.40,384.00,529.200,20.7,26.60,19.70,415.00,463.430]
test_array = np.array(test).reshape(1,-1)

In [35]:
rf.predict(test_array)

array([0], dtype=int64)